In [1]:
import pickle

non_pdf_pickle = open("google_results_non_pdf.pkl", "rb")

urls = sorted(
    list(set([res["items"][0]["link"] for res in pickle.load(non_pdf_pickle)]))
)
print(len(urls))
urls

708


['http://academiccatalog.saintleo.edu/mime/media/21/741/2016-2017+Graduate+Academic+Catalog+(Archived).pdf',
 'http://bible.ovu.edu/terry/sylbi408.htm',
 'http://bulletin.dom.edu/content.php?catoid=8&navoid=1193',
 'http://catalog.berea.edu/en/Current/Student-Handbook/Student-Rights-and-Responsibilities/Network-Usage-Guidelines',
 'http://catalog.gbc.edu/content.php?catoid=14&navoid=752',
 'http://catalog.howardcollege.edu/content.php?catoid=16&navoid=681',
 'http://catalog.miller-motte.edu/macon/',
 'http://catalog.neumann.edu/content.php?catoid=11&navoid=320',
 'http://catalog.northernvermont.edu/content.php?catoid=26&navoid=489',
 'http://catalog.smcvt.edu/content.php?catoid=32&navoid=626',
 'http://cnweb.cn.edu/ittraining2/networkusage.html',
 'http://helpdesk.hiram.edu/acceptable-use-policy/',
 'http://it.lagrange.edu/responsibleUsePolicy.php',
 'http://it.mercer.edu/student/hardware_software/it_access_and_use_policy.htm',
 'http://people.morrisville.edu/~reymers/infospace/papers/

In [2]:
import re
from funcs import *

urls_filtered = negative_filter(urls)
print(len(urls_filtered))
urls_filtered

491


['http://bible.ovu.edu/terry/sylbi408.htm',
 'http://cnweb.cn.edu/ittraining2/networkusage.html',
 'http://helpdesk.hiram.edu/acceptable-use-policy/',
 'http://it.lagrange.edu/responsibleUsePolicy.php',
 'http://people.morrisville.edu/~reymers/infospace/papers/Chicken%20Killers%20-%20IJTv2i3.pdf',
 'http://racernet.murraystate.edu/tech.htm',
 'http://wifi.fresno.edu/policy.html',
 'http://wireless.utep.edu/helppage.htm',
 'http://www.avc.edu/acceptable-use-agreement',
 'http://www.buffalo.edu/administrative-services/policy1/ub-policy-lib/ub-resnet-acceptable-use.html',
 'http://www.buffalo.edu/news/releases/2021/09/018.html',
 'http://www.bumc.bu.edu/it/policies/',
 'http://www.fullerton.edu/it/announcements/untitled.php',
 'http://www.hunter.cuny.edu/it/security/policies',
 'http://www.montana.edu/resnet/acceptable_use_policy.html',
 'http://www.saintmarys.edu/resnet/aup.html',
 'http://www.southeastern.edu/resources/policies/policy_detail/safe_driver.html',
 'http://www.tsu.edu/about

In [3]:
urls_positive = positive_filter(urls_filtered, pdf=False)
print(len(urls_positive))
urls_positive

176


['http://cnweb.cn.edu/ittraining2/networkusage.html',
 'http://helpdesk.hiram.edu/acceptable-use-policy/',
 'http://wireless.utep.edu/helppage.htm',
 'http://www.avc.edu/acceptable-use-agreement',
 'http://www.buffalo.edu/administrative-services/policy1/ub-policy-lib/ub-resnet-acceptable-use.html',
 'http://www.montana.edu/resnet/acceptable_use_policy.html',
 'http://www.saintmarys.edu/resnet/aup.html',
 'http://www.tsu.edu/about/administration/office-of-information-technology/internet-and-networking/index.html',
 'http://www.wiu.edu/policies/wireless.php',
 'https://blc.edu/campus-life/campus-services/information-technology-services/network-and-internet-access/',
 'https://case.edu/utech/departments/information-security/policies/i-1-acceptable-use-information-technology-policy-aup',
 'https://cft.vanderbilt.edu/guides-sub-pages/wireless/',
 'https://community.pepperdine.edu/law/events/networkuse/',
 'https://compliance.gwu.edu/acceptable-use-computing-systems-and-services',
 'https://

In [13]:
import requests, time
from bs4 import BeautifulSoup
from selenium import webdriver
import nltk
import random

driver = webdriver.Chrome()
blacklist = [
    "[document]",
    "noscript",
    "header",
    "html",
    "meta",
    "head",
    "input",
    "script",
    "style",
]

In [14]:
test_urls = random.sample(urls_positive, 2)
nltk.download('punkt', quiet=True)
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")

In [15]:
from funcs import *

import os

try:
    os.makedirs("AUPs/HTML")
except FileExistsError:
    pass

for url in urls_positive:
    print(url)
    school = re.search(r"(?<=[\./])\.*[\w-]+\.edu", url).group(0)
    driver.get(url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source)
    text_elements = soup.find_all(text=True)
    filtered_elements = []
    for e in text_elements:
        if e.parent.name not in blacklist:
            e = str(e).strip()
            if len(e) > 0:
                filtered_elements.append(e)
    joined_text = "\n".join(filtered_elements)
    sentences = tokenizer.tokenize(joined_text)
    fixed_sents = fix_sentences(sentences)
    with open("AUPs/HTML/" + school + ".txt", "w") as outfile:
        print(school)
        for s in fixed_sents:
            outfile.write(s + "\n")
    # print(*fixed_sents,sep="\n\n")
    print("---------------------------------", end="\n\n")

http://cnweb.cn.edu/ittraining2/networkusage.html
cn.edu
---------------------------------

http://helpdesk.hiram.edu/acceptable-use-policy/
hiram.edu
---------------------------------

http://wireless.utep.edu/helppage.htm
utep.edu
---------------------------------

http://www.avc.edu/acceptable-use-agreement
avc.edu
---------------------------------

http://www.buffalo.edu/administrative-services/policy1/ub-policy-lib/ub-resnet-acceptable-use.html
buffalo.edu
---------------------------------

http://www.montana.edu/resnet/acceptable_use_policy.html
montana.edu
---------------------------------

http://www.saintmarys.edu/resnet/aup.html
saintmarys.edu
---------------------------------

http://www.tsu.edu/about/administration/office-of-information-technology/internet-and-networking/index.html
tsu.edu
---------------------------------

http://www.wiu.edu/policies/wireless.php
wiu.edu
---------------------------------

https://blc.edu/campus-life/campus-services/information-technology-s

austincollege.edu
---------------------------------

https://www.belmont.edu/lits/policies/policies-full/acceptable-use-policy.html
belmont.edu
---------------------------------

https://www.bemidjistate.edu/offices/its/support/wired-and-wireless-networks/
bemidjistate.edu
---------------------------------

https://www.binghamton.edu/its/about/organization/operations-and-infrastructure/networking/eduroam.html
binghamton.edu
---------------------------------

https://www.biola.edu/network-usage-policy
biola.edu
---------------------------------

https://www.boisestate.edu/policy/information-technology/network/
boisestate.edu
---------------------------------

https://www.bol.ucla.edu/policies/aupdetail.html
ucla.edu
---------------------------------

https://www.bradley.edu/sites/it/help/servicedesk/services/networks/
bradley.edu
---------------------------------

https://www.brynmawr.edu/lits/about/policies/acceptable-use-policy
brynmawr.edu
---------------------------------

https://w

potsdam.edu
---------------------------------

https://www.purdue.edu/securepurdue/it-policies-standards/it-standards/wireless-use-standards.php
purdue.edu
---------------------------------

https://www.scu.edu/is/technology-policies-procedures-and-standards/information-resources-acceptable-use-agreement-/
scu.edu
---------------------------------

https://www.ship.edu/about/offices/technology/policies/acceptable-use/
ship.edu
---------------------------------

https://www.skidmore.edu/it/policies/acceptable-use.php
skidmore.edu
---------------------------------

https://www.smith.edu/its/policies/acceptable_use_policy.html
smith.edu
---------------------------------

https://www.smith.edu/its/policies/wireless_policy.html
smith.edu
---------------------------------

https://www.smu.edu/Policy/8-Information-Technology/8-1-Acceptable-Use
smu.edu
---------------------------------

https://www.stevenson.edu/about/campus-services/office-information-technology/wireless/
stevenson.edu
------